In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PSO import PSO
from Bees import BeesAlgorithm
from FireFlies import FireflyAlgorithm
import imageio
import os
import tempfile
from mpl_toolkits.mplot3d import Axes3D
temp_dir = tempfile.gettempdir()
from datetime import datetime


In [2]:
def Rastrigin(X, A=10):
    x = np.asarray(X)
    n = len(x)
    return A * n + np.sum(x**2 - A * np.cos(2 * np.pi * x))

Rastrigin_limits = [[-5.12, 5.12]]

def Rozenbrock1(X):
    x, y, = X
    f1 = (x-1)**3 - y + 1 < 0
    f2 = x + y - 2 < 0
    if all([f1, f2]):
        return (1 - x)**2 + 100*(y - x**2)**2
    else:
        return np.finfo(float).max

Rozenbrock1_limits = [[-1.5, 1.5], [-0.5, 2.5]]

def Rozenbrock2(X):
    x, y, = X
    f1 = x**2+y**2 < 2
    if f1:
        return (1 - x)**2 + 100*(y - x**2)**2
    else:
        return np.finfo(float).max

Rozenbrock2_limits = [[-1.5, 1.5], [-1.5, 1.5]]

def Mishri_Berda(X):
    x, y = X
    f1 = (x+5)**2+(y+5)**2 < 25
    if f1:
        return np.exp((1-np.cos(x))**2)*np.sin(y) + np.exp((1-np.sin(y))**2)*np.cos(x) + (x-y)**2
    else:
        return np.finfo(float).max

Mishri_Berda_limits = [[-10, 0], [-6.5, 0]]

def Siminonesku(X):
    x, y, = X
    f1 = x**2+y**2 < (1 + 0.2*np.cos(8*np.arctan(x/y)))**2
    if f1:
        return 0.1*x*y
    else:
        return np.finfo(float).max


Siminonesku_limits = [[-1.25, 1.25], [-1.25, 1.25]]

def Reductor(X):
    x1, x2, x3, x4, x5, x6, x7, = X
    x3 = int(np.clip(round(x3), 17, 28))
    
    f1 = 27 / (x1 * x2**2 * x3) - 1  <= 0
    f2 = 397.5 / (x1 * x2**2 * x3**2) - 1 <= 0
    f3 = 1.93 * x4**3 / (x2 * x3 * x6**4) - 1 <= 0
    f4 = 1.93 / (x2 * x3 * x7**4) - 1 <= 0
    f5 = 1.0/(110 * x6**3) * np.sqrt(((745*x4) / (x2 * x3))**2 + 16.9 * 10**6) - 1 <= 0
    f6 = 1.0/(85 * x7**3) * np.sqrt(((745*x5) / (x2 * x3))**2 + 157.5 * 10**6) - 1 <= 0
    f7 = (x2*x3) / 40 - 1 <= 0
    f8 = 5*x2 / x1 - 1 <= 0
    f9 = x1 / (12 * x2) - 1 <= 0
    f10 = (1.5 * x6 + 1.9) / x4 - 1 <= 0
    f11 = (1.1 * x7 + 1.9) / x5 - 1 <= 0
    if all([f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11]):
        return 0.7854*x1*x2**2*(3.3333*x3**2 + 14.9334*x3 - 43.0934) - 1.508*x1*(x6**2 + x7**2) + 7.4777*(x6**3 + x7**3) + 0.7854*(x4*x6**2 + x5*x7**2)
    return np.finfo(float).max

Reductor_limits = [[2.6, 3.6], [0.7, 0.8], [17, 28], [7.3, 8.3], [7.8, 8.3], [2.9, 3.9], [5.0, 5.5]]

def Trail(X):
    x1, x2, x3, = X
    f1 = 1-(x2**3*x3)/(7.178*x1**4) <= 0
    f2 = (4*x2**2-x1*x2)/(12.566*(x2*x1**3) - x1**4) + 1/(5.108*x1**2) - 1 <= 0
    f3 = 1 - (140.45*x1)/(x2**2*x3) <= 0
    f4 = (x2+x1)/(1.5) - 1 <= 0
    if f1 and f2 and f3 and f4:
        return (x3 + 2)*x2*x1**2
    else:
        return np.finfo(float).max
    
Trail_limits = [[0.005, 2.0], [0.25, 1.3], [2.0, 15.0]]


In [34]:
def plot_optimization_summary_gif(func, bounds, best_history, all_history, extremum=None, filename="optimization_summary.gif", ):
    dim = len(bounds)
    generations = len(best_history)
    frames = []
    temp_dir = tempfile.gettempdir()

    fig = plt.figure(figsize=(14, 10))
    if dim == 2:
        ax_func = fig.add_subplot(2, 3, 1, projection='3d')
    else:
        ax_func = fig.add_subplot(2, 3, 1)
    ax_info     = fig.add_subplot(2, 3, 2)
    ax_contour  = fig.add_subplot(2, 3, 3)
    ax_curve    = fig.add_subplot(2, 3, 4)
    ax_curve20  = fig.add_subplot(2, 3, 5)
    ax_distance = fig.add_subplot(2, 3, 6)

    for gen in range(generations):
        ax_func.clear()
        ax_info.clear()
        ax_contour.clear()
        ax_curve.clear()
        ax_curve20.clear()
        ax_distance.clear()
        
        ax = ax_func
        # === Subplot 1: Function Plot (1D or 2D surface) ===
        if dim == 1:
            x = np.linspace(bounds[0][0], bounds[0][1], 1000)
            y = np.array([func(np.array([val])) for val in x])
            ax.plot(x, y, label='Objective function')
            positions = all_history[gen]
            ax.scatter(positions[:, 0], [func(p) for p in positions], alpha=0.4, s=15, color='black')
            best_point = best_history[gen]
            ax.scatter(best_point[0], best_point[1], color='yellow', edgecolor='yellow', s=30, label='Best', zorder=1)
            ax.set_title("Function Plot (1D)")
            ax.set_xlabel("x")
            ax.set_ylabel("f(x)")

        elif dim == 2:
            X = np.linspace(bounds[0][0], bounds[0][1], 100)
            Y = np.linspace(bounds[1][0], bounds[1][1], 100)
            X, Y = np.meshgrid(X, Y)
            Z = np.apply_along_axis(func, 1, np.c_[X.ravel(), Y.ravel()]).reshape(X.shape)

            Z[np.abs(Z) > 1e100] = np.nan
                
            ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.7)
            positions = all_history[gen]
            z_vals = np.array([func(p) for p in positions])
            ax.scatter(positions[:, 0], positions[:, 1], z_vals, color='red', s=15, alpha=0.4)
            best_point = best_history[gen][0]
            best_point = best_history[gen][0]
            best_z = func(best_point)
            ax.scatter(best_point[0], best_point[1], best_z, color='yellow', edgecolor='black', s=60, label='Best', zorder=5)
            ax.set_title("Function Plot")
            ax.set_xlabel("x")
            ax.set_ylabel("y")
        else:
            ax.text(0.5, 0.5, "No visualization for dim > 2", ha='center', va='center')
            ax.set_title("Unsupported Dimension")
            ax.axis('off')

        # === Subplot 2: Info Panel ===
        ax = ax_info
        info = f"Name of function: {func.__name__}\n"
        info += f"Iteration: {gen+1}/{generations}\n"
        info += f"Best value founded: {best_history[gen][1]:.7f}\n"
        info += f"Best theoretical value: {extremum}\n"
        
        if extremum is not None:
            info += f"Distance to extremum: {abs(best_history[gen][1] - extremum):.5e}"
        ax.text(0.1, 0.5, info, fontsize=12, verticalalignment='center')
        ax.axis('off')
        ax.set_title("Info Panel")

        # === Subplot 3: Contour Plot (only for 2D) ===
        ax = ax_contour
        if dim == 2:
            ax.contourf(X, Y, Z, levels=50, cmap='plasma')
            positions = all_history[gen]
            ax.scatter(positions[:, 0], positions[:, 1], color='red', s=15, alpha=0.4)
            best_point = best_history[gen][0]
            ax.scatter(best_point[0], best_point[1], color='yellow', edgecolor='black', s=30, label='Best', zorder=1)   
            ax.set_title("Contour Plot")
        else:
            ax.text(0.5, 0.5, "Not applicable", ha='center', va='center', fontsize=14)
            ax.set_title("Contour Plot")
        ax.set_xticks([])
        ax.set_yticks([])

        # === Subplot 4: Convergence === 
        ax = ax_curve
        best_fitnesses = [score for _, score in best_history[:gen+1]]
        ax.plot(range(gen+1), best_fitnesses, marker='o', markersize=2)
        ax.set_title("Best Fitness per Iteration")
        ax.set_xlabel("Iteration")
        ax.set_ylabel("Fitness")

        # === Subplot 5: Convergence (last 20 epochs) === 
        ax = ax_curve20
        best_fitnesses = [score for _, score in best_history[:gen+1]]

        start_idx = max(0, gen - 19)
        last_epochs = list(range(start_idx, gen + 1))
        last_scores = best_fitnesses[start_idx:gen + 1]

        ax.plot(last_epochs, last_scores, marker='o',markersize=2)
        ax.set_xticks(last_epochs)
        ax.set_title("Fitness Zoomed (Last 20)")
        ax.set_xlabel("Iteration")
        ax.set_ylabel("Fitness")

        # === Subplot 6: Distance from Optimum ===
        ax = ax_distance
        if extremum is not None:
            distances = [abs(score - extremum) for _, score in best_history[:gen+1]]
            ax.plot(range(gen+1), distances, marker='o', color='red', markersize=2)
            ax.set_title("Distance to Optimum")
            ax.set_xlabel("Iteration")
            ax.set_ylabel("|f_best - f_opt|")
        else:
            ax.text(0.5, 0.5, "Extremum not provided", ha='center', va='center', fontsize=14)
            ax.set_title("Distance to Optimum")

        plt.tight_layout()
        temp_file = os.path.join(temp_dir, f"frame_{gen:03d}.png")
        fig.savefig(temp_file)
        frames.append(imageio.v2.imread(temp_file))
        plt.close(fig)

    imageio.mimsave(filename, frames, fps=10)

    for gen in range(generations):
        os.remove(os.path.join(temp_dir, f"frame_{gen:03d}.png"))

    return filename

In [ ]:
functions = [
    # (Функція, Межі, Розмірності, Теоретичний мінімум)
    (Rastrigin, Rastrigin_limits, [2, 5, 10, 15], 0.0),
    (Mishri_Berda, Mishri_Berda_limits, [2], -106.7645367),
    (Siminonesku, Siminonesku_limits, [2], -0.072),
    (Reductor, Reductor_limits, [7], None), 
    (Trail, Trail_limits, [3], None)
]

algorithms = ["pso", "bees", "firefly"]


for func, limits, dims, extremum in functions:
    for dim in dims:
        print(f"\n=== Оптимізація функції {func.__name__} розмірності {dim} ===")

        for algo in algorithms:
            print(f"  > Алгоритм: {algo.upper()}")
            max_iter_dim = np.max([75 * dim, 500])
            num_agents = np.max([50*dim, 250])
            if algo == "firefly":
                # === Вибір параметрів для Firefly ===
                alpha_firefly = 0.2
                beta0_firefly = 1.0
                gamma_firefly = 1.0
                if func.__name__ == "Rastrigin" and dim in [2, 5, 10, 15]:
                    if dim == 15:
                        alpha_firefly = 0.5
                        beta0_firefly = 1.0
                        gamma_firefly = 0.5
                    else:
                        alpha_firefly = 0.4
                        beta0_firefly = 1.0
                        gamma_firefly = 1.0
                elif func.__name__ in ["Reductor", "Trail"]:
                    alpha_firefly = 0.3
                    beta0_firefly = 1.0
                    gamma_firefly = 1.0
                elif func.__name__ in ["Mishri_Berda", "Siminonesku", "Rozenbrock1", "Rozenbrock2"]:
                    alpha_firefly = 0.4
                    beta0_firefly = 1.0
                    gamma_firefly = 1.0

                model = FireflyAlgorithm(
                    fitness_func=func,
                    num_fireflies=num_agents,
                    dim=dim,
                    pos_bounds=np.tile(limits, (dim, 1)) if len(limits) == 1 else limits,
                    max_iter=max_iter_dim,
                    alpha=alpha_firefly,
                    beta0=beta0_firefly,
                    gamma=gamma_firefly,
                    target_score=extremum,
                    epsilon_target=1e-6
                )
            elif algo == "pso":
                model = PSO(
                    fitness_func=func,
                    num_particles=num_agents,
                    dim=dim,
                    pos_bounds=np.tile(limits, (dim, 1)) if len(limits) == 1 else limits,
                    vel_bounds=[-1, 1],
                    max_iter=max_iter_dim,
                    alpha1=1.5,
                    alpha2=1.5,
                    w_max=0.9,
                    w_min=0.4,
                    target_global_min =extremum,
                    epsilon_target= 1e-6

                )
            elif algo == "bees":
                model = BeesAlgorithm(
                    fitness_func=func,
                    num_bees=num_agents,
                    dim=dim,
                    pos_bounds=np.tile(limits, (dim, 1)) if len(limits) == 1 else limits,
                    max_iter=max_iter_dim,
                    elite_sites=3,
                    best_sites=10,
                    elite_bees=10,
                    recruited_bees=5,
                    delta_init=0.1,
                    alpha=0.95,
                    eta_max=0.9,
                    target_score=extremum,
                    epsilon_target=1e-6
                    
                )
            else:
                raise ValueError(f"Алгоритм {algo} не підтримується.")

            # Запуск алгоритму
            best_pos, best_score, history_best, history_all_positions = model.run()

            print(f"    Найкраща позиція: {best_pos}")
            print(f"    Найкращий результат: {best_score}")

            # Створення GIF
            timestamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
            filename = f"gif_results/{func.__name__}_d{dim}_{algo}_summary_{timestamp}.gif"

            plot_optimization_summary_gif(
                func=func,
                bounds=np.tile(limits, (dim, 1)) if len(limits) == 1 else limits,
                best_history=history_best,
                all_history=[np.array(pos) for pos in history_all_positions],
                extremum=extremum,
                filename=filename
            )
            print(f"    Збережено GIF: {filename}")



=== Оптимізація функції Mishri_Berda розмірності 2 ===
  > Алгоритм: PSO
Iteration 0: Global best score = -104.56558056773846
Iteration 10: Global best score = -106.76315716620132
Iteration 20: Global best score = -106.76315716620132
Iteration 30: Global best score = -106.76342511174292
Iteration 40: Global best score = -106.76377706409312
Iteration 50: Global best score = -106.7645312607794
Global minimum is within epsilon 1e-06 of target at iteration 54. Stopping.
    Найкраща позиція: [-3.13031986 -1.58211511]
    Найкращий результат: -106.76453591160546
    Збережено GIF: gif_results/Mishri_Berda_d2_pso_summary_27-04-2025_21-57-43.gif
  > Алгоритм: BEES
Global minimum is within epsilon 1e-06 of target at iteration 125. Stopping.
    Найкраща позиція: [-3.13021323 -1.58217229]
    Найкращий результат: -106.7645364678857
    Збережено GIF: gif_results/Mishri_Berda_d2_bees_summary_27-04-2025_21-58-48.gif
  > Алгоритм: FIREFLY
Global minimum is within epsilon 1e-06 of target at iterat

C:\Users\grund\AppData\Local\Temp\ipykernel_17160\1684034385.py:126: UserWarning: Tight layout not applied. tight_layout cannot make Axes width small enough to accommodate all Axes decorations
  plt.tight_layout()
d:\University\semestr_6\venv_6semestr\Lib\site-packages\matplotlib\axes\_base.py:3054: RuntimeWarning: overflow encountered in scalar add
  x0, x1 = inverse_trans.transform([x0t - delta, x1t + delta])


    Збережено GIF: gif_results/Reductor_d7_bees_summary_27-04-2025_22-17-21.gif
  > Алгоритм: FIREFLY
    Найкраща позиція: [ 3.5         0.7        17.00046132  7.45716566  8.14363165  3.35056139
  5.28680317]
    Найкращий результат: 3005.442014609154
    Збережено GIF: gif_results/Reductor_d7_firefly_summary_27-04-2025_22-23-46.gif

=== Оптимізація функції Trail розмірності 3 ===
  > Алгоритм: PSO
Iteration 0: Global best score = 2.2247192728770844
Iteration 10: Global best score = 2.2247192728770844
Iteration 20: Global best score = 2.1988321920306113
Iteration 30: Global best score = 2.1690229539803942
Iteration 40: Global best score = 2.166482816919984
Iteration 50: Global best score = 2.1661956869949637
Iteration 60: Global best score = 2.134139683533782
Iteration 70: Global best score = 2.0952093363980326
Iteration 80: Global best score = 2.0897739776804896
Iteration 90: Global best score = 2.0893986796050448
Iteration 100: Global best score = 2.0893986796050448
Iteration 110: 

<Figure size 640x480 with 0 Axes>

In [6]:
import pandas as pd
functions = [
    (Rastrigin, Rastrigin_limits, [2, 5, 10, 15], 0.0),
    (Mishri_Berda, Mishri_Berda_limits, [2], -106.7645367),
    (Siminonesku, Siminonesku_limits, [2], -0.072),
    (Reductor, Reductor_limits, [7], None), 
    (Trail, Trail_limits, [3], None)
]

algorithms = ["pso", "bees", "firefly"]

for func, limits, dims, extremum in functions:
    print(f"\n=== Оптимізація функції {func.__name__} ===")
    records = [] 

    for dim in dims:
        print(f" -> Розмірність {dim}")

        for algo in algorithms:
            max_iter_dim = np.min([75 * dim, 500])
            num_agents = np.min([50 * dim, 250])

            for run in range(3):
                if algo == "firefly":
                    alpha_firefly = 0.2
                    beta0_firefly = 1.0
                    gamma_firefly = 1.0
                    if func.__name__ == "Rastrigin" and dim in [2, 5, 10, 15]:
                        if dim == 15:
                            alpha_firefly = 0.5
                            beta0_firefly = 1.0
                            gamma_firefly = 0.5
                        else:
                            alpha_firefly = 0.4
                            beta0_firefly = 1.0
                            gamma_firefly = 1.0
                    elif func.__name__ in ["Reductor", "Trail"]:
                        alpha_firefly = 0.3
                        beta0_firefly = 1.0
                        gamma_firefly = 1.0
                    elif func.__name__ in ["Mishri_Berda", "Siminonesku", "Rozenbrock1", "Rozenbrock2"]:
                        alpha_firefly = 0.4
                        beta0_firefly = 1.0
                        gamma_firefly = 1.0

                    model = FireflyAlgorithm(
                        fitness_func=func,
                        num_fireflies=num_agents,
                        dim=dim,
                        pos_bounds=np.tile(limits, (dim, 1)) if len(limits) == 1 else limits,
                        max_iter=max_iter_dim,
                        alpha=alpha_firefly,
                        beta0=beta0_firefly,
                        gamma=gamma_firefly,
                        target_score=extremum,
                        epsilon_target=1e-6,
                        stagnation_iter=25
                    )
                elif algo == "pso":
                    model = PSO(
                        fitness_func=func,
                        num_particles=num_agents,
                        dim=dim,
                        pos_bounds=np.tile(limits, (dim, 1)) if len(limits) == 1 else limits,
                        vel_bounds=[-1, 1],
                        max_iter=max_iter_dim,
                        alpha1=1.5,
                        alpha2=1.5,
                        w_max=0.9,
                        w_min=0.4,
                        target_score=extremum,
                        epsilon_target=1e-6
                    )
                elif algo == "bees":
                    model = BeesAlgorithm(
                        fitness_func=func,
                        num_bees=num_agents,
                        dim=dim,
                        pos_bounds=np.tile(limits, (dim, 1)) if len(limits) == 1 else limits,
                        max_iter=max_iter_dim,
                        elite_sites=3,
                        best_sites=10,
                        elite_bees=10,
                        recruited_bees=5,
                        delta_init=0.1,
                        alpha=0.95,
                        eta_max=0.9,
                        target_score=extremum,
                        epsilon_target=1e-6
                    )
                else:
                    raise ValueError(f"Алгоритм {algo} не підтримується.")

                best_pos, best_score, history_best, history_all_positions = model.run()

                # Додаємо запис у список
                records.append({
                    "Function": func.__name__,
                    "Dimension": dim,
                    "Algorithm": algo.upper(),
                    "Run": run + 1,
                    "Best Score": best_score
                })

    # === Створення DataFrame для поточної функції ===
    df_func = pd.DataFrame(records)

    print(f"\n=== Результати для функції {func.__name__} ===")
    print(df_func)

    filename = f"results_{func.__name__}.csv"
    df_func.to_csv(filename, index=False)
    print(f"Результати збережено у файл: {filename}")


=== Оптимізація функції Rastrigin ===
 -> Розмірність 2
Iteration 0: Global best score = 1.0047906011464107
Iteration 10: Global best score = 0.9984457601630261
Iteration 20: Global best score = 0.023117531045645734
Iteration 30: Global best score = 0.0029536101441323126
Iteration 40: Global best score = 0.0003579228451542349
Iteration 50: Global best score = 9.443922596119592e-05
Iteration 60: Global best score = 5.703164447368181e-06
Global minimum is within epsilon 1e-06 of target at iteration 68. Stopping.
Iteration 0: Global best score = 2.028757295819428
Iteration 10: Global best score = 0.10884830231151454
Iteration 20: Global best score = 0.0009616912688912294
Iteration 30: Global best score = 0.0009616912688912294
Iteration 40: Global best score = 0.00035825021148383485
Iteration 50: Global best score = 1.4773546610058474e-05
Iteration 60: Global best score = 1.6355418352986817e-06
Global minimum is within epsilon 1e-06 of target at iteration 62. Stopping.
Iteration 0: Global

In [7]:
Mishri_Berda_df = pd.read_csv("results_Mishri_Berda.csv")
Mishri_Berda_df

,Function,Dimension,Algorithm,Run,Best Score
0,Mishri_Berda,2,PSO,1,-106.764537
1,Mishri_Berda,2,PSO,2,-106.764536
2,Mishri_Berda,2,PSO,3,-106.764536
3,Mishri_Berda,2,BEES,1,-106.764536
4,Mishri_Berda,2,BEES,2,-106.764536
5,Mishri_Berda,2,BEES,3,-106.764536
6,Mishri_Berda,2,FIREFLY,1,-106.764520
7,Mishri_Berda,2,FIREFLY,2,-106.759630
8,Mishri_Berda,2,FIREFLY,3,-106.764536


In [3]:
dim = 10
pso = PSO(
    fitness_func=Rastrigin, 
    num_particles=400, 
    dim=dim, 
    pos_bounds = Rastrigin_limits*dim, 
    vel_bounds = [-1, 1],
    max_iter=1000,
    alpha1=2.0, 
    alpha2=1.0,
    w_max=0.9, 
    w_min=0.5,
)

best_pos, best_score, history_best, history_all_positions = pso.run()

Iteration 0: Global best score = 72.70343646968837
Iteration 10: Global best score = 23.575925584532158
Iteration 20: Global best score = 14.341611034820957
Iteration 30: Global best score = 6.655933590185015
Iteration 40: Global best score = 5.750702033388393
Iteration 50: Global best score = 5.563639348412295
Iteration 60: Global best score = 5.212342838747972
Iteration 70: Global best score = 5.119934615355973
Iteration 80: Global best score = 5.0583969593399445
Iteration 90: Global best score = 5.036792685521277
Iteration 100: Global best score = 5.025813563744293
Iteration 110: Global best score = 5.00539510747609
Iteration 120: Global best score = 4.9900131571189945
Iteration 130: Global best score = 4.985878797028036
Iteration 140: Global best score = 4.510374911326267
Iteration 150: Global best score = 3.1519214836634433
Iteration 160: Global best score = 3.083967349593081
Iteration 170: Global best score = 3.048661261316198
Iteration 180: Global best score = 3.007587704132092


In [4]:
dim = 2
pso = PSO(
    fitness_func=Rozenbrock2, 
    num_particles=100, 
    dim=dim, 
    pos_bounds=Rozenbrock1_limits, 
    vel_bounds=[-1, 1],
    max_iter=300,
    alpha1=2.0, 
    alpha2=1.0,
    w_max=0.9, 
    w_min=0.4,
)

best_pos, best_score, history_best, history_all_positions = pso.run()

Iteration 0: Global best score = 0.15405866213794814
Iteration 10: Global best score = 0.004479308561597605
Iteration 20: Global best score = 0.000911334149395388
Iteration 30: Global best score = 0.000911334149395388
Iteration 40: Global best score = 0.000911334149395388
Iteration 50: Global best score = 0.0001261564113316228
Iteration 60: Global best score = 5.392964363466232e-05
Iteration 70: Global best score = 1.1375151328723112e-05
Iteration 80: Global best score = 2.2447630977900224e-07
Iteration 90: Global best score = 2.2447630977900224e-07
Iteration 100: Global best score = 2.2447630977900224e-07
Iteration 110: Global best score = 1.764013921184749e-08
Iteration 120: Global best score = 3.477766566379802e-09
Iteration 130: Global best score = 8.821984080405965e-12
Iteration 140: Global best score = 8.821984080405965e-12
Iteration 150: Global best score = 1.0973788398607279e-12
Iteration 160: Global best score = 3.257067619371249e-16
Iteration 170: Global best score = 3.257067

In [8]:
dim = 7
pso = PSO(
    fitness_func=Reductor, 
    num_particles=300, 
    dim=dim, 
    pos_bounds=Reductor_limits, 
    vel_bounds=[-1, 1],
    max_iter=500,
    alpha1=2.0, 
    alpha2=1.0,
    w_max=0.9, 
    w_min=0.4,
)

best_pos, best_score, history_best, history_all_positions = pso.run()
print(best_pos)

Iteration 0: Global best score = 3561.660901586198
Iteration 10: Global best score = 3271.5731466135885
Iteration 20: Global best score = 3271.5731466135885
Iteration 30: Global best score = 3271.5731466135885
Iteration 40: Global best score = 3141.96840980787
Iteration 50: Global best score = 3141.96840980787
Iteration 60: Global best score = 3069.8512186977387
Iteration 70: Global best score = 3046.9548174793313
Iteration 80: Global best score = 3043.2887939333405
Iteration 90: Global best score = 3043.2887939333405
Iteration 100: Global best score = 3022.230961555456
Iteration 110: Global best score = 3011.5681948457227
Iteration 120: Global best score = 3011.5681948457227
Iteration 130: Global best score = 3009.802644403605
Iteration 140: Global best score = 3002.880546375379
Iteration 150: Global best score = 2998.0026868986715
Iteration 160: Global best score = 2997.5405691753044
Iteration 170: Global best score = 2996.815985746578
Iteration 180: Global best score = 2996.63350983

In [6]:
dim = 3
pso = PSO(
    fitness_func=Trail, 
    num_particles=500,    
    dim=dim, 
    pos_bounds=Trail_limits, 
    vel_bounds=[-1, 1],  
    max_iter=500,          
    alpha1=1.5, 
    alpha2=1.5,
    w_max=0.9, 
    w_min=0.4
)

best_pos, best_score, history_best, history_all_positions = pso.run()

print(f"Best found position: {best_pos}")
print(f"Best found score: {best_score}")

Iteration 0: Global best score = 2.9567975131164856
Iteration 10: Global best score = 2.381132128261832
Iteration 20: Global best score = 2.280913747130291
Iteration 30: Global best score = 2.146576307256832
Iteration 40: Global best score = 2.115813626931046
Iteration 50: Global best score = 2.1003573552368597
Iteration 60: Global best score = 2.0986392989461797
Iteration 70: Global best score = 2.088804035935951
Iteration 80: Global best score = 2.0854202517380087
Iteration 90: Global best score = 2.0802282656494437
Iteration 100: Global best score = 2.0801405798389054
Iteration 110: Global best score = 2.0794827324694527
Iteration 120: Global best score = 2.079371505934662
Iteration 130: Global best score = 2.079371505934662
Iteration 140: Global best score = 2.0792753158108703
Iteration 150: Global best score = 2.0791853551507393
Iteration 160: Global best score = 2.0791581447263
Iteration 170: Global best score = 2.079154576286506
Iteration 180: Global best score = 2.0791509260257

In [7]:
dim = 3
bees = BeesAlgorithm(
    fitness_func=Trail,          
    num_bees=200,
    dim=dim,
    pos_bounds=Trail_limits,
    max_iter=500,
    elite_sites=3,
    best_sites=10,
    elite_bees=10,
    recruited_bees=5,
    delta_init=0.1, 
    alpha=0.95
)

best_pos, best_score, history_best, history_all_positions = bees.run()

print(f"Best position: {best_pos}")
print(f"Best score: {best_score}")

Iteration 0: Global best score = 2.446232914456813
Iteration 10: Global best score = 2.1974013652234676
Iteration 20: Global best score = 2.1974013652234676
Iteration 30: Global best score = 2.1974013652234676
Iteration 40: Global best score = 2.1974013652234676
Iteration 50: Global best score = 2.1807702847421155
Iteration 60: Global best score = 2.1252949705615998
Iteration 70: Global best score = 2.1038031385279803
Iteration 80: Global best score = 2.084695347908553
Iteration 90: Global best score = 2.084695347908553
Iteration 100: Global best score = 2.084695347908553
Iteration 110: Global best score = 2.084695347908553
Iteration 120: Global best score = 2.084695347908553
Iteration 130: Global best score = 2.084695347908553
Iteration 140: Global best score = 2.084695347908553
Iteration 150: Global best score = 2.084695347908553
Iteration 160: Global best score = 2.084695347908553
Iteration 170: Global best score = 2.084695347908553
Iteration 180: Global best score = 2.0846953479085

In [8]:
dim = 2
firefly = FireflyAlgorithm(
    fitness_func=Rastrigin,
    num_fireflies=100,
    dim=dim,
    pos_bounds=Rastrigin_limits * dim,
    max_iter=300,
    alpha=0.5,
    beta0=0.8,
    gamma=0.8,
    
)

best_pos, best_score, history_best, history_all_positions = firefly.run()

print(f"Best position: {best_pos}")
print(f"Best score: {best_score}")

Iteration 0: Global best score = 4.881438894191584
Iteration 10: Global best score = 1.0004091027120197
Iteration 20: Global best score = 1.0004091027120197
Iteration 30: Global best score = 1.0004091027120197
Iteration 40: Global best score = 1.0004091027120197
Iteration 50: Global best score = 0.706738910861052
Iteration 60: Global best score = 0.011994777499118925
Iteration 70: Global best score = 0.011994777499118925
Iteration 80: Global best score = 0.00038853711812691927
Iteration 90: Global best score = 0.00038853711812691927
Iteration 100: Global best score = 0.00038853711812691927
Iteration 110: Global best score = 0.00038853711812691927
Iteration 120: Global best score = 0.0003234349248160129
Iteration 130: Global best score = 0.00012634344737350034
Iteration 140: Global best score = 2.225664868760191e-05
Iteration 150: Global best score = 2.1064908040102637e-07
Iteration 160: Global best score = 2.1064908040102637e-07
Iteration 170: Global best score = 2.1064908040102637e-07

In [9]:
dim = 15 
firefly = FireflyAlgorithm(
    fitness_func=Rastrigin,
    num_fireflies=100,
    dim=dim,
    pos_bounds=Rastrigin_limits * dim,
    max_iter=300,
    alpha=0.5,
    beta0=1,
    gamma=0.5
)

best_pos, best_score, history_best, history_all_positions = firefly.run()

print(f"Best position: {best_pos}")
print(f"Best score: {best_score}")

Iteration 0: Global best score = 190.97768546820956
Iteration 10: Global best score = 145.85879202063734
Iteration 20: Global best score = 136.4771055772788
Iteration 30: Global best score = 136.4771055772788
Iteration 40: Global best score = 136.4771055772788
Iteration 50: Global best score = 136.4771055772788
Iteration 60: Global best score = 136.4771055772788
Iteration 70: Global best score = 136.4771055772788
Iteration 80: Global best score = 136.4771055772788
Iteration 90: Global best score = 128.30125325995115
Iteration 100: Global best score = 128.30125325995115
Iteration 110: Global best score = 128.30125325995115
Iteration 120: Global best score = 128.30125325995115
Iteration 130: Global best score = 128.30125325995115
Iteration 140: Global best score = 128.30125325995115
Iteration 150: Global best score = 128.30125325995115
Iteration 160: Global best score = 128.30125325995115
Iteration 170: Global best score = 128.30125325995115
Iteration 180: Global best score = 128.3012532

In [14]:
dim = 15
pso = PSO(
    fitness_func=Rastrigin, 
    num_particles=300, 
    dim=dim, 
    pos_bounds = Rastrigin_limits*dim, 
    vel_bounds = [-1, 1],
    max_iter=500,
    alpha1=1.0, 
    alpha2=2.0,
    w_max=0.9, 
    w_min=0.5,
)

best_pos, best_score, history_best, history_all_positions = pso.run()

Iteration 0: Global best score = 148.12801814322134
Iteration 10: Global best score = 35.66465362541818
Iteration 20: Global best score = 22.09722919213054
Iteration 30: Global best score = 15.417210234569694
Iteration 40: Global best score = 14.711817710351482
Iteration 50: Global best score = 9.459784292764425
Iteration 60: Global best score = 9.365470206701247
Iteration 70: Global best score = 9.365470206701247
Iteration 80: Global best score = 9.26506761688671
Iteration 90: Global best score = 9.20432315975691
Iteration 100: Global best score = 9.11328115742046
Iteration 110: Global best score = 9.07325590216493
Iteration 120: Global best score = 8.993973855827647
Iteration 130: Global best score = 8.973897222609224
Iteration 140: Global best score = 8.966711644761034
Iteration 150: Global best score = 8.960933004686012
Iteration 160: Global best score = 8.958406981536399
Iteration 170: Global best score = 8.957314634238799
Iteration 180: Global best score = 8.956012471811277
Itera